## reading CSV files with Pandas and some simple Dataframe manipulations

In [ ]:
%matplotlib inline

### pathlib
import pathlib

### scipy
import numpy as np
import pandas as pd

### plotting
from matplotlib import pyplot as plt
import matplotlib

In [ ]:
import pathlib

In [ ]:
plt.rcParams['figure.figsize'] = (10,5)
plt.rcParams['font.size'] = 14
plt.rcParams['lines.linewidth'] = 2

#### Daily temperature data for 30 NZ stations from https://data.mfe.govt.nz/table/105056-daily-temperature-1909-2019/ 

In [ ]:
dpath = pathlib.Path('/media/nicolasf/END19101/data/NZ_station_data/temperature/')

#### read the data in csv, setting up the index (labels along the rows) to be the first column (indexing starts at 0 in Python)

In [ ]:
data = pd.read_csv(dpath.joinpath('daily-temperature-1909-2019.csv'), index_col=0)

In [ ]:
data.head()

In [ ]:
location_names = data.index.unique()

In [ ]:
location_names

In [ ]:
len(location_names)

### select a location (index) and a statistic in the `statistic` column 

In [ ]:
location = 'Hamilton'

In [ ]:
statistic = 'Maximum'

In [ ]:
df = data.loc[location,:].query(f"statistic == '{statistic}'")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df = df.set_index('date')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.index

In [ ]:
df.index = pd.to_datetime(df.index)

In [ ]:
df.index

In [ ]:
df = df[['temperature']]

In [ ]:
df.head()

In [ ]:
df.plot()

### resampling, take the maximum daily temperature for each calendar month

In [ ]:
df_monthly = df.resample('1M').max()

In [ ]:
df_monthly.plot()

In [ ]:
df_monthly.describe()

In [ ]:
df.describe()

### calculate the climatological 90th percentile (value exceeded only 10% of the time)

In [ ]:
clim = df_monthly.loc['1990':'2020',:]

In [ ]:
clim_q90 = clim.groupby(clim.index.month).quantile(0.90)

In [ ]:
clim_q90

In [ ]:
clim_q90.plot()

### repeat these values along the whole time-series 

In [ ]:
clim_q90 = clim_q90.loc[df_monthly.index.month,:]

In [ ]:
clim_q90.head()

In [ ]:
len(clim_q90)

In [ ]:
len(df_monthly)

In [ ]:
df_monthly.loc[:,'q90'] = clim_q90.values

In [ ]:
df_monthly.head()

### we can now do some comparisons 

In [ ]:
df_monthly.loc[:,'temperature'] >= df_monthly.loc[:,'q90']

In [ ]:
hot_months = df_monthly.loc[df_monthly.temperature >= df_monthly.q90,'temperature']

In [ ]:
hot_months.head()

In [ ]:
len(hot_months)

### use the `reindex` method to create a continuous time-series (NaNs (Not a Number, i.e. missing values) will be inserted)

In [ ]:
hot_months = hot_months.reindex(df_monthly.index)

In [ ]:
hot_months.plot(marker='o', linewidth=0)